In [1]:
results_page = 'https://www.sheffield.gov.uk/your-city-council/city-council-parish-council-election-results'

In [3]:
import requests
from bs4 import BeautifulSoup

In [4]:
resp = requests.get(results_page)

In [8]:
soup = BeautifulSoup(resp.text)

In [21]:
fields = soup.findAll('div',{'class':'field field--name-localgov-text field--type-text-long field--label-hidden field__item'})

In [123]:
elections = {}
election_name = election_result = None
for field in fields:
    if field.find('h2') is not None:
        elections[election_name] = election_result

        election_name = field.find('h2').text

        election_result = {}
    elif field.find('h4') is not None:
        ward = field.find('h4').text
        election_result[ward] = {}

        elec_info = field.find('p').text.split('\n')

        for pt in elec_info:
            pt_name , pt_val = pt.split(':')

            election_result[ward][pt_name] = pt_val

        elec_results = field.find('ul').findAll('li')

        cand_votes = {}

        for result in elec_results:
            cand, party, votes = process_result(result.text)
            if party == 'Independent':
                party = f'{party} - {cand}'
            cand_votes[party] = votes

        election_result[ward]['detail'] = cand_votes



elections[election_name] = election_result


190 votes



In [120]:
import subprocess

def copy2clip(txt):
    cmd='echo '+txt.strip()+'|clip'
    return subprocess.check_call(cmd, shell=True)

def process_result(res):
    res = res.replace(':',',')
    if len(res.split(',')) <3:
        res= res.replace('Party', 'Party,')

    i=0
    candidate = res.split(',')[i]
    i+=1

    poss_parties = {
        'Green Party':'Green',
        'Conservative Party':'Conservative',
        'Conservative Party Candidate':'Conservative',
        'Trade Unionist and Socialist Coalition':'TUSC',
        'Liberal Democrats':'Lib Dem',
        'Labour and Co-operative Party':'Labour',
        'Yorkshire Party':'Yorkshire',
        'Labour Party':'Labour',
        'Labour Part':'Labour',
        'Northern Independence Party - Nationalise Energy Companies':'NIP',
        "Women's Equality Party":'WEP',
        'Social Democratic Party':'SDP',
        'Communist Party of Britain':'Communist',
        'Independent':'Independent',
        'UK Independence Party (UKIP)':'UKIP',
        'Reform UK':'Reform',
        '':'',
        '':'',
        '':'',
    }

    party = res.split(',')[i].strip()
    if ' votes' in party:
        print(party)
        party = 'Independent'
        i-=1

    if party not in poss_parties:
        copy2clip(party)
        i+=1
        party = res.split(',')[i].strip()


    if party not in poss_parties:
        raise ValueError(res)

    party= poss_parties[party]

    votes = ''.join(result.text.split(',')[i+1:])
    try:
        votes = int(votes.replace('votes','').strip())
    except:
        print(votes)

    return candidate, party, votes




In [124]:
elections

{None: None,
 'Elections 5 May 2022': {},
 '5 May 2022': {'Park and Arbourthorne': {'Elected': ' Fathima Nabeela Mowlana, Labour and Co-operative Party',
   'Total votes': ' 3,329',
   'Rejected ballots': ' 27',
   'Electorate': '\xa013,180',
   'Turnout': '\xa025.26 %',
   'detail': {'Green': 575,
    'Conservative': 608,
    'TUSC': 106,
    'Lib Dem': 275,
    'Labour': 1479,
    'Yorkshire': 259}},
  'Beauchief and Greenhill': {'Elected': '\xa0Simon William Clement-Jones,\xa0Liberal Democrats',
   'Total votes': ' 4,904',
   'Rejected ballots': ' 40',
   'Electorate': ' 13,835',
   'Turnout': '\xa035.45%',
   'detail': {'Labour': 1769,
    'Lib Dem': 1951,
    'Conservative': 579,
    'Green': ''}},
  'Beighton': {'Elected': ' Kurtis Jenson Crossland, Liberal Democrats',
   'Total votes': ' 4,053\xa0',
   'Rejected ballots': ' 20',
   'Electorate': '\xa013,295',
   'Turnout': '\xa030.49%',
   'detail': {'Lib Dem': 1582,
    'Labour': 1498,
    'Green': 298,
    'Conservative': 655}